In [ ]:
#change this to the correct path
data_path = 'C:/Users/adloose/Downloads/WindTurbines_Sirris'

In [ ]:
#import packages
import os
import math
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

In [ ]:
#split data into weather and tech 
def AD_split(data_WT):
    data_WT_tech=np.zeros((data_WT.shape[0], 96),)
    data_WT_tech[:, 0:64]=data_WT[:, 0:64]
    data_WT_tech[:, 64:]=data_WT[:, 80:112]
    data_WT_weather=np.zeros((data_WT.shape[0], 28),)
    data_WT_weather[:, 0:16]=data_WT[:, 64:80] #Ws1, Ws2, Ws and Ot
    data_WT_weather[:, 16:]=data_WT[:, 112:] #Wa and Va (angles)
    return(data_WT_tech, data_WT_weather)

In [ ]:
#read in data
data_WT1 = pd.read_csv(data_path+'/WT1_full.csv', sep=';')
data_WT1 = data_WT1.values #numpy array
data_WT2 = pd.read_csv(data_path+'/WT2_full.csv', sep=';')
data_WT2 = data_WT2.values #numpy array
data_WT3 = pd.read_csv(data_path+'/WT3_full.csv', sep=';')
data_WT3 = data_WT3.values #numpy array
data_WT4 = pd.read_csv(data_path+'/WT4_full.csv', sep=';')
data_WT4 = data_WT4.values #numpy array

#list variables
var_list = pd.read_csv(data_path+'/var_list.csv', sep=';', header=None)
var_list=list(var_list.values)
for i in range(len(var_list)): #list creates array at each position
    var_list[i]=var_list[i][0]
    
#split list
var_list_tech=var_list[0:64]+var_list[80:112]
var_list_weather=var_list[64:80]+var_list[112:]

In [ ]:
#split data in weather and technical variables
(data_WT1_tech, data_WT1_weather)=AD_split(data_WT1)
(data_WT2_tech, data_WT2_weather)=AD_split(data_WT2)
(data_WT3_tech, data_WT3_weather)=AD_split(data_WT3)
(data_WT4_tech, data_WT4_weather)=AD_split(data_WT4)

In [ ]:
#use a linear regression to predict each of the other variables
#predict 96 tech variables based on 28 weather variables
Y=np.zeros((data_WT1_tech.shape[0],),) #output
r_sq=np.zeros((data_WT1_tech.shape[1],),) #save the r squares
intercepts= np.zeros((data_WT1_tech.shape[1],),) #intercepts
coefficients=np.zeros((data_WT1_tech.shape[1], data_WT1_weather.shape[1]),) #coefficients
data_WT1_pred=np.zeros((data_WT1_tech.shape[0], data_WT1_tech.shape[1]),) #predictions 1
data_WT2_pred=np.zeros((data_WT1_tech.shape[0], data_WT1_tech.shape[1]),) #predictions 2
data_WT3_pred=np.zeros((data_WT1_tech.shape[0], data_WT1_tech.shape[1]),) #predictions 3
data_WT4_pred=np.zeros((data_WT1_tech.shape[0], data_WT1_tech.shape[1]),) #predictions 4

model_LR=LinearRegression()
for i in range(data_WT1_tech.shape[1]):
    Y=data_WT1_tech[:, i]
    model_LR.fit(data_WT1_weather, Y)
    r_sq[i]=model_LR.score(data_WT1_weather, Y)
    intercepts[i]=model_LR.intercept_
    coefficients[i, :]=model_LR.coef_
    data_WT1_pred[:, i]=model_LR.predict(data_WT1_weather)

#other turbines
for i in range(data_WT1_tech.shape[1]):
    Y=data_WT2_tech[:, i]
    model_LR.fit(data_WT2_weather, Y)
    data_WT2_pred[:, i]=model_LR.predict(data_WT2_weather)
    
    Y=data_WT3_tech[:, i]
    model_LR.fit(data_WT3_weather, Y)
    data_WT3_pred[:, i]=model_LR.predict(data_WT3_weather)
    
    Y=data_WT4_tech[:, i]
    model_LR.fit(data_WT4_weather, Y)
    data_WT4_pred[:, i]=model_LR.predict(data_WT4_weather)

In [ ]:
r_sq.mean()

In [ ]:
#uncomment to see the whole list of r2 values

#for i in range(data_WT1_tech.shape[1]):
    #print(var_list_tech[i] + ': ' +str(r_sq[i]))

In [ ]:
#calculate adjusted data
data_WT1_tech2=data_WT1_tech-data_WT1_pred #substract predicted data
data_WT2_tech2=data_WT2_tech-data_WT2_pred
data_WT3_tech2=data_WT3_tech-data_WT3_pred
data_WT4_tech2=data_WT4_tech-data_WT4_pred

In [ ]:
#plot some variables to check if it's working properly
x=range(52560) #one year
a=0

plt.plot(x, data_WT1_tech[0:52560, a], 'o', label='Original')
plt.plot(x, data_WT1_tech2[0:52560, a], '-', label='Weather corrected')
plt.legend()
plt.title('Rt')
plt.show()
plt.close()

a=32

plt.plot(x, data_WT1_tech[0:52560, a], 'o', label='Original')
plt.plot(x, data_WT1_tech2[0:52560, a], '-', label='Weather corrected')
plt.legend()
plt.title('Dbt1')
plt.show()
plt.close()
